# Paper Savior 2

Auto-explorative research with **Retrieval Augumented Generation(RAG)** and **Reason-Action prompting (ReAct)**, using agent framework `lionagi` with `llama_index` as ToolBox

let us conduct a research session. We will, 
- download 20 papers from arxiv as our primary inspiration
- give our researcher 5 abstracts to **read** and propose some ideas to explore
- ask researcher to **look up** relevant terms and information from **5 sources**
- draft plans and points and present final research proposal 


**WARNING** : This notebook uses `gpt-4-turbo-preview` for workflow, and can get ***very expensive***

You can:
1. change workflow model, but that will require context length being managed
2. Reduce the number of steps and the number of queries

In [1]:
# %pip install lionagi llama-index llama_hub unstructured pypdf arxiv wikipedia google-search 'unstructured[pdf]'

In [2]:
# if you would like to ignore logging

import logging
logging.getLogger().setLevel(logging.ERROR)

In [3]:
topic = "Large Language Model Applications in Blockchain"
question = "Research on building a system of trust integrating Large Language Model with blockchain"
num_papers = 20

persist_dir = ".storage/"

Note: only works with llama_index legacy

## 1. Setup Data QA Tools

#### a. ArXiv Index

We will download papers using our research topic from **ArXiv**, a popular pre-publish platform for research papers

with `LlamaIndex` and build it into a searchable index.

- [ArXiv Official Website](https://arxiv.org)

- [LlamaIndex Official Website](https://www.llamaindex.ai)

- [ArXivReader on LlamaHub](https://llamahub.ai/l/papers-arxiv?from=all) 

In [4]:
from llama_index import download_loader
from llama_index import  ServiceContext, VectorStoreIndex
from llama_index.text_splitter import SentenceSplitter
from llama_index.llms import OpenAI
from llama_index.embeddings import OpenAIEmbedding

## get arxiv papers using llamaindex ArxivReader
ArxivReader = download_loader("ArxivReader")
loader = ArxivReader()

documents, abstracts = loader.load_papers_and_abstracts(
    search_query=topic, max_results=num_papers)

# set up service context for LlamaIndex  
text_splitter = SentenceSplitter(chunk_size=2048, chunk_overlap=50)         # split the documents into chunks with whole sentences
llm = OpenAI(model='gpt-4-turbo-preview', temperature=0.1)                  # set up the llm to be used
embedding = OpenAIEmbedding(model='text-embedding-3-small')

service_context = ServiceContext.from_defaults(
    text_splitter=text_splitter, llm=llm)

arxiv_index = VectorStoreIndex.from_documents(
    documents, service_context=service_context)

arxiv_engine = arxiv_index.as_query_engine(
    similarity_top_k=3, response_mode= "tree_summarize")

arxiv_index.storage_context.persist(f'{persist_dir}arxiv1/')

If you have already built an index and stored it by running above codes, you can `index object` build from storage

you can build from storage using the following, just need to find the `index_id` in the index_store file

- but you **still need** have some abstract of paper or other things as the starting context for `Session`

In [5]:
# from llama_index import StorageContext, load_index_from_storage
# from llama_index.llms import OpenAI

# llm = OpenAI(model='gpt-4-turbo-preview', temperature=0.1)
# service_context = ServiceContext.from_defaults(llm=llm)

# storage_context = StorageContext.from_defaults(persist_dir=f'{persist_dir}arxiv/')
# # <change to your own index id, can find it in index store>
# index_id = 'c3ce7613-5fc1-42d1-aa95-968e878ad2db'

# arxiv_index = load_index_from_storage(
#     storage_context=storage_context, 
#     index_id=index_id, 
#     service_context=service_context
# )

# arxiv_engine = arxiv_index.as_query_engine(similarity_top_k=3, response_mode= "tree_summarize")

#### b. Textbooks

use a couple pdf textbooks as references and build a Knowledge Graph Index

- [Dive into Deep Learning](https://d2l.ai)

- [Blockchain for Dummies - IBM edition](http://gunkelweb.com/coms465/texts/ibm_blockchain.pdf)

- [KnowledgeGraphIndex](https://docs.llamaindex.ai/en/stable/examples/index_structs/knowledge_graph/KnowledgeGraphDemo.html) 


we will use unstructured reader for pdf

In [6]:
from llama_index import download_loader, Document

UnstructuredReader = download_loader("UnstructuredReader")
loader = UnstructuredReader()

In [7]:
# download a couple free textbooks online as references 

book1 = "d2l-en.pdf"
book2 = "ibm_blockchain.pdf"

docs1 = loader.load_data(book1)
docs2 = loader.load_data(book2)

documents1 = [Document(text="".join([x.text for x in docs1]))]
documents2 = [Document(text="".join([x.text for x in docs2]))]

##### Build KG

this will take quite a while, I suggest you to minimize the notebook for 0.5-1 hours, or change the source to be shorter in length

In [8]:
from llama_index import ServiceContext, KnowledgeGraphIndex
from llama_index.graph_stores import SimpleGraphStore
from llama_index.storage.storage_context import StorageContext
from llama_index.llms import OpenAI
from llama_index.embeddings import OpenAIEmbedding


llm = OpenAI(model='gpt-4-turbo-preview', temperature=0.1) 
embedding = OpenAIEmbedding(model='text-embedding-3-large')
service_context = ServiceContext.from_defaults(llm=llm, chunk_size=512, embed_model=embedding)
storage_context = StorageContext.from_defaults(graph_store=SimpleGraphStore())

In [9]:
d2l_index = KnowledgeGraphIndex.from_documents(
    documents1,
    max_triplets_per_chunk=2,
    storage_context=storage_context,
    service_context=service_context,
    include_embeddings=True,
)

d2l_engine = d2l_index.as_query_engine(similarity_top_k=3, response_mode= "tree_summarize")
d2l_index.storage_context.persist(f'{persist_dir}d2l/')

In [10]:
bc_index = KnowledgeGraphIndex.from_documents(
    documents2,
    max_triplets_per_chunk=2,
    storage_context=storage_context,
    service_context=service_context,
    include_embeddings=True,
)

bc_engine = bc_index.as_query_engine(similarity_top_k=3, response_mode= "tree_summarize")
bc_index.storage_context.persist(f'{persist_dir}bc_ibm/')

##### From Storage

In [11]:
# from llama_index import load_index_from_storage
# from llama_index import ServiceContext
# from llama_index.graph_stores import SimpleGraphStore
# from llama_index.storage.storage_context import StorageContext
# from llama_index.llms import OpenAI
# from llama_index.embeddings import OpenAIEmbedding


# llm = OpenAI(model='gpt-4-turbo-preview', temperature=0.1) 
# embedding = OpenAIEmbedding(model='text-embedding-3-large')
# service_context = ServiceContext.from_defaults(llm=llm, embed_model=embedding)
# storage_context = StorageContext.from_defaults(graph_store=SimpleGraphStore())

In [12]:
# storage_context = StorageContext.from_defaults(
#     graph_store=SimpleGraphStore(), persist_dir= f'{persist_dir}/d2l/')

# # <change to your own index id, can find it in index store>
# index_id = '7c52b76a-bd85-4aa8-a167-ab4f468b7dc9' 

# d2l_index = load_index_from_storage(storage_context=storage_context, index_id=index_id, service_context=service_context)
# d2l_engine = d2l_index.as_query_engine(similarity_top_k=3, response_mode= "tree_summarize")

In [13]:
# storage_context = StorageContext.from_defaults(
#     graph_store=SimpleGraphStore(), persist_dir= f'{persist_dir}/bc_ibm/')

# # <change to your own index id, can find it in index store>
# index_id = '4850f9e9-158c-44d3-a3f0-1bfec72dfc6e' 

# bc_index = load_index_from_storage(storage_context=storage_context, index_id=index_id, service_context=service_context)
# bc_engine = bc_index.as_query_engine(similarity_top_k=3, response_mode= "tree_summarize")

#### c. google and wikipedia

we will use Google and Wikipedia to clarify certain domain specific terms with LlamaIndex `OpenAI agent`

in order to use google search engine you will have to register with google and get an `API_KEY` and a `google_engine`

- [Instruction on Getting Google Search API and Engine](https://developers.google.com/custom-search/v1/overview)
- [Google Tools on LlamaHub](https://llamahub.ai/l/tools-google_search?from=all)
- [Wiki Tools on LlamaHub](https://llamahub.ai/l/tools-wikipedia?from=all)

Once after you get the API_KEY and Search_Engine save them to `.env` file, 

GOOGLE_API_KEY='...'

GOOGLE_CSE_ID='...'

In [14]:
import os

google_key_scheme = 'GOOGLE_API_KEY'
google_engine_scheme = 'GOOGLE_CSE_ID'

In [15]:
llm = OpenAI(model='gpt-4-turbo-preview', temperature=0.1)

# we will create agents for google search and wikipedia querying
def create_google_agent(
    google_api_key=os.getenv(google_key_scheme), 
    google_engine=os.getenv(google_engine_scheme), 
    verbose=False
):
    from llama_index.agent import OpenAIAgent
    from llama_index.tools.tool_spec.load_and_search.base import LoadAndSearchToolSpec
    from llama_hub.tools.google_search.base import GoogleSearchToolSpec

    api_key = google_api_key
    search_engine = google_engine
    google_spec = GoogleSearchToolSpec(key=api_key, engine=search_engine)

    # Wrap the google search tool as it returns large payloads
    tools = LoadAndSearchToolSpec.from_defaults(
        google_spec.to_tool_list()[0],
    ).to_tool_list()

    agent = OpenAIAgent.from_tools(tools, verbose=verbose, llm=llm)
    return agent

def create_wiki_agent(verbose=False):
    from llama_hub.tools.wikipedia import WikipediaToolSpec
    from llama_index.agent import OpenAIAgent

    tool_spec = WikipediaToolSpec()

    agent = OpenAIAgent.from_tools(tool_spec.to_tool_list(), verbose=verbose, llm=llm)
    return agent

# 2. Tools

Now we have set up the needed components for tools, let us set up tools for LLM to use. 


1. write a function definition for the tool, and add google style docstring
2. keep track of the responses for source checking
3. make these functions into `lionagi.Tool` object

we will define the functions as `asynchorous` functions so we can run parallel queries concurrently 

In [16]:
responses_arxiv = []
responses_d2l = []
responses_bc = []

async def query_arxiv(query: str):
    """
    Query a vector index built with papers from arxiv. It takes 
    natural language query, and give natural language response. 

    Args:
        query (str): The natural language query to get an answer from the index

    Returns:
        str: The query response from index
    """
    response = await arxiv_engine.aquery(query)
    responses_arxiv.append(response)
    
    return str(response.response)

async def query_d2l(query: str):
    """
    Query a index built from machine learning textbooks. It takes 
    natural language query, and give natural language response. 

    Args:
        query (str): The natural language query to get an answer from the index

    Returns:
        str: The query response from index
    """
    response = await d2l_engine.aquery(query)
    responses_d2l.append(response)
    
    return str(response.response)
        
async def query_bc(query: str):
    """
    Query a index built from blockchain textbooks. It takes 
    natural language query, and give natural language response. 

    Args:
        query (str): The natural language query to get an answer from the index

    Returns:
        str: The query response from index
    """
    response = await bc_engine.aquery(query)
    responses_bc.append(response)
    
    return str(response.response)

In [17]:
responses_google = []
responses_wiki = []

# ask gpt to write you google format docstring
async def query_google(query: str):
    """
    Search Google and retrieve a natural language answer to a given query.

    Args:
        query (str): The search query to find an answer for.

    Returns:
        str: A natural language answer obtained from Google search results.

    Raises:
        Exception: If there is an issue with making the request or parsing the response.
    """
    google_agent = create_google_agent()
    response = await google_agent.achat(query)
    responses_google.append(response)
    return str(response.response)

async def query_wiki(query: str):
    """
    Search Wikipedia and retrieve a natural language answer to a given query.

    Args:
        query (str): The search query to find an answer for.

    Returns:
        str: A natural language answer obtained from Google search results.

    Raises:
        Exception: If there is an issue with making the request or parsing the response.
    """
    wiki_agent = create_wiki_agent()
    response = await wiki_agent.achat(query)
    responses_wiki.append(response)
    return str(response.response)

# 3. Workflow and Outputs

Make the functions into LionAGI `Tool` objects

the `func_to_tool` function converts a function with google or rest style docstring into a `Tool` object, 

which can be used during a `Session`

In [18]:
import lionagi as li

funcs = [query_google, query_wiki, query_arxiv, query_d2l, query_bc]

# lcall is a loop handler, it apply a function to every element in the input list
tools = li.lcall(funcs, li.func_to_tool)

In [19]:
# prompts
system = {
    "persona": "you are a helpful assistant, perform as a researcher",
    "notice": f"your research topic is on {topic}, and the question is {question}",
    "requirements": "Think step by step",
    "responsibilities": "Researching a specific topic and question, explore and provide specific findings and insights",
    "deliverable": "technical only, ~ 1000-1500 words, briefly explain the core concepts and rationale behind, retain from being vague or general, target audience is highly sophisticated and can judge your work. "
}

instruct = f"""
read a few paper abstracts, carefully propose a few unique, creative, pratical and achieveable solutions to 
solve the research question on the specific topics, notice you can use the query tools in parallel, but your questions need to be all different and specific to the tools. Your final deliverable needs to be highly specific and technical. you have to use every tool at least once, but as extensively as you can. 
"""

contexts = str(abstracts[:4])

### a. First attempt

In [20]:
# set up a researcher session
researcher = li.Session(system)

# provide the tools
researcher.register_tools(tools)

# invoke the task for researcher via reason-action technique
out = await researcher.ReAct(instruct, context=contexts, num_rounds=3, temperature=0.7)

In [21]:
from IPython.display import Markdown
Markdown(out)

I'm unable to execute further tool usage actions directly. However, the next steps in the research project have been outlined clearly in the action plan. If you have any specific requests or need further assistance, please let me know how I can help!

In [22]:
researcher.messages

,node_id,role,sender,timestamp,content
0,87b892051e997e4f66d907fb1c62def1,system,system,2024-02-11T12:00:07.812855,"{""system_info"": {""persona"": ""you are a helpful..."
1,6fd3c0aea74fa446e9b55256573bd892,user,user,2024-02-11T12:00:07.813497,"{""instruction"": {""Notice"": "" \n ..."
2,33a42318a5fbedd5bd82873997cc2677,assistant,assistant,2024-02-11T12:00:44.586011,"{""response"": ""To address the research question..."
3,d8ce576aa8620209f8501efb0b018896,user,user,2024-02-11T12:00:44.588026,"{""instruction"": ""\n you have 5 ..."
4,b6c7607f2783f6117ce64d51b78917dd,assistant,action_request,2024-02-11T12:00:53.092398,"{""action_list"": [{""action"": ""action_query_arxi..."
5,b90a93ac4cd10cea7b72a44a2c363395,assistant,action_response,2024-02-11T12:01:31.080254,"{""action_response"": {""function"": ""query_arxiv""..."
6,9ec68d8712511d5f8368d9b9bf42ae32,assistant,action_response,2024-02-11T12:01:31.081321,"{""action_response"": {""function"": ""query_wiki"",..."
7,ccc12a738e7e5294f2411fc96a310d64,assistant,action_response,2024-02-11T12:01:31.081757,"{""action_response"": {""function"": ""query_google..."
8,4bce6147b6779195ccd3f806a803313e,assistant,action_response,2024-02-11T12:01:31.082219,"{""action_response"": {""function"": ""query_d2l"", ..."
9,ad202966ef30f0e5cdc7b18f8eb87b04,assistant,action_response,2024-02-11T12:01:31.082614,"{""action_response"": {""function"": ""query_bc"", ""..."


seem like the assistant gave answer during the ReAct, instead of presenting outcome in the end, let us check what it did

In [30]:
df = researcher.messages
answer = df[df.sender == "assistant"].content.iloc[-2]

In [31]:
Markdown(li.as_dict(answer)['response'])

Reflecting on the gathered information and moving forward with the action plan, we enter the next phase of our research project. The insights provided through the use of various tools highlight both the potential and the challenges in integrating Large Language Models (LLMs) with blockchain to create trust systems. Each tool has offered a unique perspective, contributing to a multifaceted understanding of how LLMs can enhance blockchain technology and vice versa.

### Step 1: Technical Deep Dive into Proposed Solutions

Based on the synthesized insights, we identified four key areas where LLMs can significantly impact blockchain technology, specifically regarding reliability prediction, interoperability, time management, and the deployment and management of blockchain applications. Here, we will outline a technical deep dive for each proposed solution:

1. **LLM-Aided Peer Reliability Assessment System**:
   - **Objective**: Develop a system that utilizes LLMs to assess and predict the reliability of blockchain peers or nodes based on historical transaction data and peer interactions.
   - **Technical Approach**: Implement NLP techniques to analyze qualitative feedback and performance metrics. Use supervised learning models trained on historical data to predict reliability scores for peers. Integrate this system with blockchain networks to dynamically update peer reliability scores.
   
2. **Smart Contract Translator for Interoperability**:
   - **Objective**: Create a tool that leverages LLMs to translate smart contracts from one blockchain platform's language to another, enhancing interoperability.
   - **Technical Approach**: Train an LLM on a diverse dataset of smart contracts across different blockchain languages. Implement a translation mechanism that maintains the logical integrity and security properties of the original contract. Evaluate the translated contracts for performance and security.

3. **Temporal Logic Handler for Smart Contracts**:
   - **Objective**: Develop an LLM module that introduces temporal logic into smart contracts, enabling them to handle time-based conditions and events more naturally.
   - **Technical Approach**: Use LLMs to parse and modify smart contracts, incorporating time-related constructs and logic. Explore the integration of blockchain-specific time measures for accurate event handling.

4. **Blockchain Application Management Suite with LLM**:
   - **Objective**: Build a suite of tools that utilize LLMs for automated code generation, security auditing, and user interface creation for blockchain applications.
   - **Technical Approach**: Develop an LLM-based code generator for smart contracts and blockchain applications. Create an anomaly detection system for identifying security vulnerabilities. Implement an LLM-powered interface generator that simplifies user interaction with blockchain applications.

### Step 2: Implementation Strategy

For each solution, the implementation strategy involves several key steps:

- **Dataset Collection and Preparation**: Collect and preprocess data relevant to each solution, including smart contracts, blockchain transaction data, and user feedback.
- **Model Training and Tuning**: Train LLMs using the prepared datasets, fine-tuning them for specific tasks such as translation, reliability prediction, and code generation.
- **Integration with Blockchain Technologies**: Develop APIs and interfaces for integrating the LLM-based solutions with existing blockchain platforms and applications.
- **Validation and Testing**: Conduct extensive testing and validation to ensure the effectiveness, security, and performance of the proposed solutions.
- **Deployment and Monitoring**: Deploy the solutions in real-world scenarios, monitor their performance, and iteratively improve based on user feedback and observed outcomes.

By following this detailed action plan, we aim to advance the integration of LLMs with blockchain technology, addressing key challenges and leveraging their combined strengths to enhance trust systems.

### b. Improve work

In [23]:
out1 = await researcher.chat(
    """
    you asked a lot of good questions and got plenty answers, please integrate your 
    conversation, be a lot more technical, you will be rewarded with 500 dollars for 
    great work, and punished for subpar work, take a deep breath, you can do it
    """
)

In [24]:
Markdown(out1)

Integrating insights from our comprehensive exploration of integrating Large Language Models (LLMs) with blockchain technology to build trust systems, we delve into a technical synthesis aimed at proposing viable, innovative solutions. This synthesis is informed by the rich information extracted from various sources, including academic papers, interoperability frameworks, and foundational principles of blockchain technology. Our focus zeroes in on four pivotal areas: reliability prediction of blockchain peers, interoperability enhancements, time management in blockchain processes, and the deployment and management of blockchain applications.

### Reliability Prediction Using LLMs

The use of LLMs for predicting the reliability of blockchain peers or nodes is an untapped area with substantial potential. While traditional methods, as highlighted in the Hybrid Blockchain Reliability Prediction (H-BRP) model, rely on historical data and central data collectors for prediction, integrating LLMs could revolutionize this approach. By leveraging NLP techniques, LLMs can analyze unstructured data, such as peer reviews or qualitative feedback, alongside quantitative performance metrics. This dual analysis approach could yield a more nuanced and accurate prediction model.

**Technical Approach**:
- **Data Collection**: Aggregate historical transaction data, peer interaction logs, and qualitative feedback.
- **Model Training**: Employ supervised learning to train LLMs on this aggregated data, focusing on identifying patterns correlated with high reliability.
- **Integration**: Develop an API for blockchain networks that utilizes the trained LLM to dynamically update reliability scores based on ongoing peer performance and feedback.

### Enhancing Interoperability with LLMs

Interoperability remains a significant challenge within the blockchain ecosystem. The insights gathered suggest that LLMs could play a crucial role in translating smart contracts between different blockchain languages, thus facilitating seamless interaction across platforms.

**Technical Approach**:
- **Dataset Creation**: Compile a comprehensive dataset of smart contracts from various blockchain platforms.
- **LLM Training**: Train an LLM on this dataset to recognize and translate between different blockchain languages while preserving logical and security properties.
- **Deployment**: Develop a platform-agnostic tool that leverages the trained LLM for smart contract translation, offering it as a service to blockchain developers.

### Temporal Logic in Smart Contracts

The absence of a natural notion of time in smart contracts is a gap that LLMs could fill. Through the integration of temporal logic, smart contracts could handle time-based conditions more effectively.

**Technical Approach**:
- **Logic Definition**: Define a set of temporal logic constructs tailored for blockchain applications.
- **LLM Modification**: Train LLMs to parse and modify existing smart contracts to incorporate these constructs, ensuring that the contracts can interact with blockchain-specific time measures accurately.
- **Smart Contract Development**: Encourage the adoption of these enhanced smart contracts by providing a development toolkit that includes LLM-based temporal logic integration.

### LLM-Powered Blockchain Application Management

The deployment and management of blockchain applications stand to benefit significantly from LLM integration. Automating code generation, enhancing security through anomaly detection, and simplifying user interfaces are key areas of potential impact.

**Technical Approach**:
- **Automated Code Generation**: Develop an LLM-trained on a diverse set of smart contracts to generate or suggest code snippets, reducing developer effort and increasing code reliability.
- **Anomaly Detection**: Train LLMs on transaction patterns to identify and alert about potential fraudulent activities or security breaches.
- **User Interface Generation**: Utilize LLMs to create intuitive, natural language interfaces for blockchain applications, lowering the barrier to entry for non-technical users.

### Conclusion

The integration of LLMs with blockchain technology opens a new frontier for enhancing trust systems. By addressing reliability prediction, interoperability, temporal challenges in smart contracts, and application deployment and management, LLMs can significantly contribute to the robustness, security, and user-friendliness of blockchain technologies. Future research should focus on developing specific LLM architectures and training methodologies tailored to these blockchain-centric tasks, ensuring the scalability and adaptability of these solutions to meet evolving needs and challenges within the blockchain ecosystem.

### c. check the queries

In [43]:
df = researcher.messages

print(len(df[df.sender == "action_response"]))

10


there are 10 tool usages during this ReAct session, let us take a look at some

In [56]:
res_dict1 = li.as_dict(df[df.sender == "action_response"].content.iloc[4])

func_ = li.nget(res_dict1, ['action_response', 'function'])
args_ = li.nget(res_dict1, ['action_response', 'arguments'])
output_ = li.nget(res_dict1, ['action_response', 'output'])

print(f"Tool used is: {func_}")
print(f"question asked: {args_}")
Markdown(output_)

Tool used is: query_bc
question asked: {'query': 'What foundational principles of blockchain technology can be leveraged to integrate LLMs for creating trust systems?'}


The foundational principles of blockchain technology that can be leveraged to integrate Large Language Models (LLMs) for creating trust systems include:

1. **Distributed and Sustainable**: The ledger is shared, updated with every transaction, and selectively replicated among participants in near real time. This principle ensures that the blockchain platform's existence isn't dependent on any single organization, which can be applied to LLMs by distributing the model's knowledge base across a network, ensuring its sustainability and resilience.

2. **Secure, Private, and Indelible**: Permissions and cryptography prevent unauthorized access to the network and ensure that participants are who they claim to be. Privacy is maintained through cryptographic techniques and/or data partitioning techniques. This principle can be applied to LLMs by ensuring that interactions and data exchanges are secure and private, with an immutable record of transactions and interactions.

3. **Transparent and Auditable**: Participants in a transaction have access to the same records, allowing them to validate transactions and verify identities or ownership without the need for third-party intermediaries. This principle can be applied to LLMs by making the decision-making process transparent and auditable, allowing users to understand how conclusions are reached.

4. **Consensus-based and Transactional**: All relevant network participants must agree that a transaction is valid, achieved through the use of consensus algorithms. This principle can be applied to LLMs by incorporating consensus mechanisms for decision-making processes, ensuring that outputs are validated by multiple parties before being accepted.

5. **Orchestrated and Flexible**: Business rules and smart contracts can be built into the platform, allowing blockchain business networks to evolve to support end-to-end business processes and a wide range of activities. This principle can be applied to LLMs by enabling them to operate within a framework of smart contracts for automated execution of tasks based on predefined conditions, enhancing flexibility and orchestration of complex processes.

By leveraging these foundational principles, LLMs can be integrated into trust systems that are secure, transparent, and capable of facilitating consensus-based decision-making, thereby enhancing trust among network participants.

In [57]:
res_dict1 = li.as_dict(df[df.sender == "action_response"].content.iloc[6])

func_ = li.nget(res_dict1, ['action_response', 'function'])
args_ = li.nget(res_dict1, ['action_response', 'arguments'])
output_ = li.nget(res_dict1, ['action_response', 'output'])

print(f"Tool used is: {func_}")
print(f"question asked: {args_}")
Markdown(output_)

Tool used is: query_wiki
question asked: {'query': 'What are the existing frameworks or models for interoperability among blockchain platforms, and how can LLMs enhance them?'}


Interoperability among blockchain platforms is a critical area of development that aims to enable seamless communication and interaction between different blockchain networks. This is essential for creating a more connected and efficient blockchain ecosystem. While specific frameworks or models for blockchain interoperability were not directly detailed, we can discuss the general concept and how Large Language Models (LLMs) could potentially enhance these frameworks.

### Existing Frameworks or Models for Blockchain Interoperability

1. **Cross-Chain Technology**: This involves creating protocols that enable the transfer of assets and information between different blockchain networks. Examples include Polkadot's parachains, Cosmos' Inter-Blockchain Communication (IBC) protocol, and the Wanchain project.

2. **Blockchain Bridges**: Bridges are specific implementations of cross-chain technology that connect two blockchains, allowing for the transfer of assets and data between them. They can be trust-based, requiring intermediaries, or trustless, relying on smart contracts and decentralized mechanisms.

3. **Sidechains and Layer 2 Solutions**: Sidechains are separate blockchains that are connected to a main chain, allowing for asset transfer between the two. Layer 2 solutions, like the Lightning Network for Bitcoin, provide scalability and interoperability benefits by handling transactions off the main chain.

4. **Atomic Swaps**: This is a technology that enables the exchange of one cryptocurrency for another without the need for a trusted third party, using a mechanism called Hashed Timelock Contracts (HTLCs).

5. **Interledger Protocols**: These are protocols designed to connect different ledgers and payment systems, not limited to blockchains, to enable secure and seamless transactions across them.

### How LLMs Can Enhance Blockchain Interoperability

LLMs, with their advanced language understanding and generation capabilities, can contribute to enhancing blockchain interoperability in several ways:

1. **Smart Contract Development and Verification**: LLMs can assist in the development of more sophisticated and secure smart contracts that are crucial for interoperability solutions like blockchain bridges and atomic swaps. They can help in generating code, detecting vulnerabilities, and verifying the correctness of smart contracts.

2. **Protocol Translation**: LLMs can facilitate the translation of protocols between different blockchain networks, acting as a linguistic bridge that interprets and converts data formats, transaction models, and consensus mechanisms.

3. **Automated Governance**: For interoperability frameworks that involve governance mechanisms, LLMs can assist in automating decision-making processes, interpreting governance proposals, and generating reports or summaries to inform stakeholders.

4. **User Interaction**: LLMs can improve the user experience by providing natural language interfaces for interacting with cross-chain technologies, making it easier for non-technical users to engage with complex interoperability solutions.

5. **Documentation and Education**: LLMs can generate and maintain comprehensive documentation for interoperability frameworks, protocols, and APIs, facilitating easier adoption and understanding by developers and users alike.

In conclusion, while blockchain interoperability remains a complex challenge, the integration of LLMs into interoperability solutions can offer significant benefits in terms of security, efficiency, and usability. As both blockchain technology and LLMs continue to evolve, their synergistic potential is likely to become increasingly important for the development of a fully interconnected blockchain ecosystem.

### d. Save the work

In [32]:
researcher.messages.to_csv("researcher_messages.csv")

In [41]:
responses = li.to_list([responses_arxiv, responses_d2l, responses_bc, responses_google, responses_wiki], flatten=True)
responses_dicts = li.lcall(responses, lambda x: {"response": str(x)})

In [42]:
responses_df = li.to_df(responses_dicts)
responses_df.to_csv("research_query_responses.csv")